In [4]:
import yfinance as yf

In [27]:
msft = yf.Ticker("MSFT")
msft.history(period="1y")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2019-06-20,135.78,135.99,134.07,135.29,33042600,0.0,0
2019-06-21,134.92,136.06,134.80,135.31,36727900,0.0,0
2019-06-24,135.33,136.72,135.33,136.11,20628800,0.0,0
2019-06-25,135.58,135.92,131.12,131.81,33327400,0.0,0
2019-06-26,132.72,134.09,131.98,132.30,23657700,0.0,0
...,...,...,...,...,...,...,...
2020-06-15,184.58,190.82,184.01,188.94,32770200,0.0,0
2020-06-16,192.89,195.58,191.46,193.57,42556700,0.0,0
2020-06-17,195.03,196.32,193.69,194.24,25655900,0.0,0


In [28]:
!jupyter nbconvert --to script *.ipynb

[NbConvertApp] Converting notebook api_test.ipynb to script
[NbConvertApp] Writing 74 bytes to api_test.txt
